<a href="https://colab.research.google.com/github/Viktory4121/diploma/blob/main/cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import files
import xlrd
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
uploaded = files.upload()

Saving ConvertedTable.xlsx to ConvertedTable.xlsx


In [44]:
data = pd.read_excel(uploaded['ConvertedTable.xlsx'])
pdata = pd.DataFrame(data)
#Удаление лишних столбцов
pdata = pdata.drop('Отметка времени', axis=1)
pdata = pdata.drop('Учебное заведение', axis=1)
pdata = pdata.drop('Направление и профиль', axis=1)
pdata = pdata.drop('Курс', axis=1)

#Изменение формата представления пола студента
pdata['Мужской пол'] = [int(pdata['Пол'][i] == 'Мужской') for i in range(len(pdata))]
pdata = pdata.drop('Пол', axis=1)

pdata

,Возраст,Не нравится составленное расписание,Составленное расписание неудобно,Не устраивает работа профорганизаций,Не устраивает работа кафедры,Не устраивает работа деканата,Документы оформляют слишком долго,Недостаточное количество учебной литературы,Маленькие стипендии,"Мало конкурсов/турниров, где можно показать себя",...,Не успеваю сдать долги,Не хочу углубляться в учебу,Эта профессия не принесёт пользу обществу,Учёба мешает развиваться,Не хочу «бежать» за оценками,Нет мотивации,"Получаю образованию только для того, чтобы было",Близкие не одобряют выбор профессии,Приходится жить далеко от родных и близких,Мужской пол
0,20,6,6,6,8,5,5,3,3,7,...,10,10,7,8,10,8,10,2,2,1
1,20,10,10,5,0,0,0,5,4,0,...,0,2,0,0,0,0,0,1,0,0
2,20,1,1,0,0,1,0,4,10,4,...,2,10,0,10,10,10,10,4,0,1
3,20,5,5,4,4,4,4,4,5,4,...,3,5,5,5,3,5,5,0,1,0
4,20,0,0,5,0,3,4,0,7,0,...,0,5,0,4,5,0,9,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,24,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102,23,3,1,1,1,3,3,2,7,5,...,0,0,0,0,4,0,0,0,0,1
103,24,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104,24,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
x = pdata.copy()
kmeans_2 = KMeans(n_clusters=2, random_state=0)
kmeans_2.fit(x)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2, random_state=0)

In [61]:
predict_2cl = kmeans_2.predict(x)
predict_2cl

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)